In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
import src.utils as utils
from src.calcEMA import *
from src.myenv import *
from src.train import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
import src.utils as utils

has_results = utils.has_results('BTCUSDT', 
                        '30m', 
                        'knn', 
                        'allknn', 
                        '2010-01-01', 
                        '2023-01-01 00:30:00', 
                        'volume', 
                        48, 
                        1.0)

has_results

In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55, 32, 234], 'close': [3, 4, 5, 6], 'C': [5, 6, 34, 5], 'D': [7, 8, 54, 66], 'label': ['A', 'A', 'B', 'B']}, index=[0,1,2,3])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)

In [ ]:
_df = get_klines('BTCUSDT', interval='1h', max_date=None, limit=200)
_df

In [ ]:
from pycaret.classification.oop import ClassificationExperiment

ca = ClassificationExperiment()
setup = ca.setup(_df, target='symbol')

In [ ]:
best = setup.compare_models()

best

In [ ]:
setup.pull().index[0]

In [ ]:
import datetime 
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

In [ ]:
import pandas as pd
import datetime
import os

df_resultado_simulacao = pd.DataFrame()
if (os.path.exists('resultado_simulacao.csv')):
    df_resultado_simulacao = pd.read_csv('resultado_simulacao.csv', sep=';')

result_simulado = {}
result_simulado['model_name'] = 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1'
result_simulado['data'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
result_simulado['symbol'] = 'BTCUSDT'
result_simulado['estimator'] = 'lightgbm'
result_simulado['train_size'] = 0.7 
result_simulado['start_train_date'] = '2023-06-01'
result_simulado['start_test_date'] = '2023-08-08'
result_simulado['regression_times'] = 24*30
result_simulado['times_regression_profit_and_loss'] = 24
result_simulado['stop_loss'] = 2.0
result_simulado['fold'] = 3
result_simulado['saldo_inicial'] = 100.00
result_simulado['saldo_final'] = 91.87

print(result_simulado)

df = pd.DataFrame.from_dict([result_simulado])

df_resultado_simulacao = pd.concat([df_resultado_simulacao, df], ignore_index=True)
df_resultado_simulacao.sort_values('saldo_final', inplace=True)

df_resultado_simulacao.to_csv('resultado_simulacao.csv', sep=';', index=False)

In [ ]:
from binance.client import Client
client = Client()
klines = client.get_historical_klines(symbol='BTCUSDT', interval='1m', start_str='2023-09-01', limit=1000)
klines



In [ ]:
import pandas as pd
pd.to_datetime(1695036000000, unit='ms')

In [ ]:
import pandas as pd


all_klines_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades', 'ignore', 'close_time']

df_klines = pd.DataFrame(data=klines, columns=all_klines_cols)[columns]
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')

In [ ]:
df_klines

In [ ]:
download_data()


In [ ]:
all_data = get_data('BTCUSDT', save_database=False, interval='1h', tail=-1, columns=['open_time', 'close'])
all_data

In [ ]:
import numpy as np

at = all_data.tail(1).open_time.dt.strftime('%Y-%m-%d %H').values[0]


In [ ]:
list_of_elements = ['close','open','high','low','volume']
fix_it='close'

combinations_list = []
if len(list_of_elements) > 0:    
    for i in range(1, len(list_of_elements) + 1):
        a = combinations(list_of_elements, i)
        for s in a:
            res = ''
            for j in s:
                res += f'{j},'
            combinations_list.append(res[0:len(res) - 1])

combinations_list

In [ ]:
from itertools import combinations
import pandas as pd

comb = combinations(range(1, 25 + 1), 15)
cols = ['col_' + str(i) for i in range(1, 15 + 1)]
resultado = pd.DataFrame(list(comb), columns=cols)
resultado

In [ ]:
resultado.query('col_1 in (1,2,3,4) and col_15 in (22,23,24,25)')

In [ ]:
import pandas as pd
results = ['a','b','c','d','d','d','d','d',]

pd.DataFrame(results, columns=['status'])['status'].value_counts()

In [ ]:
import pandas as pd
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import logging

logging.basicConfig(
    level=logging.DEBUG,  # Set the minimum level to be logged
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler()])


df = pd.read_csv('../../data/ADAUSDT/ADAUSDT.csv', sep=";", index_col='open_time')
df['open_time'] = pd.to_datetime(df.index, unit='ms')
df

In [ ]:
df.sort_index(inplace=True)
df.index.duplicated().sum()

In [ ]:
all_data = utils.regression_PnL(
            df,
            label=myenv.label,
            diff_percent=1.0,
            max_regression_profit_and_loss=6,
            drop_na=True,
            drop_calc_cols=True,
            strategy=None)

all_data
            

In [ ]:
all_data['prediction_label'] = all_data['status']
all_data = utils.simule_trading_crypto(all_data, '2023-06-01', '2023-09-19', 100, 1.0)
all_data 

In [ ]:
import sys
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
import src.utils as utils
import src.myenv as myenv
import src.send_message as sm
import os
import glob
import pandas as pd



In [ ]:
symbol, interval = 'BTCUSDT', '1h'

cont = 0
cont_aviso = 101

purchased = False
purchase_price = 0.0
actual_price = 0.0
amount_invested = 0.0
# balance = 0.0
take_profit_price = 0.0
stop_loss_price = 0.0
profit_and_loss = 0.0
operation = ''
margin = 0.0
margin_operation = 0.0
rsi = 0.0
latest_closed_candle_open_time_aux = None

balance = 1000.00

params_operation = utils.get_latest_operation(symbol, interval)
print(params_operation)
if len(params_operation) > 0 and params_operation['operation'] == 'BUY':
	purchased = True
	purchase_price = float(params_operation['purchase_price'])
	take_profit_price = float(params_operation['take_profit'])
	stop_loss_price = float(params_operation['stop_loss'])
	operation = params_operation['status']
	latest_closed_candle_open_time_aux = params_operation["operation_date"]
	amount_invested = float(params_operation['amount_invested'])
	rsi = float(params_operation['rsi'])
	margin_operation = round(((take_profit_price - purchase_price) / purchase_price) * 100, 2)

	print(latest_closed_candle_open_time_aux, operation, purchase_price, amount_invested, balance, margin_operation, take_profit_price, stop_loss_price)

In [ ]:
symbol = 'TESTUSDT'
interval = '1m'
def calc_sl_pnl(operation, actual_value, margin):
	take_profit_value = 0.0
	stop_loss_value = 0.0
	if operation.startswith('CAI'):  # Short
		take_profit_value = actual_value * (1 - margin / 100)
		stop_loss_value = actual_value * (1 + (margin * myenv.stop_loss_multiplier) / 100)
	elif operation.startswith('SOBE'):  # Long
		take_profit_value = actual_value * (1 + margin / 100)
		stop_loss_value = actual_value * (1 - (margin * myenv.stop_loss_multiplier) / 100)
	return take_profit_value, stop_loss_value

def log_selling(open_time, operation, purchase_price, actual_price, margin, amount_invested, profit_and_loss, balance, take_profit_price,
								stop_loss_price, margin_operation):
	msg = f'*SELLING*: Symbol: {symbol}_{interval} - Open Time: {open_time} - Operation: {operation} - Target Margin: {margin_operation:.2f}% '
	msg += f'- Purchased Price: $ {purchase_price:.6f} - Actual Price: $ {actual_price:.6f} - Margin: {100*margin:.2f}% - Amount invested: $ {amount_invested:.2f} '
	msg += f'- PnL: $ {profit_and_loss:.2f} - Take Profit: $ {take_profit_price:.6f} - Stop Loss: $ {stop_loss_price:.6f} - Balance: $ {balance:.2f}'
	print(f'{msg}')

In [ ]:

purchased = True
operation = 'SOBE'
margin_operation = 1.5
actual_price = 1014
purchase_price = 1000
amount_invested = 100
balance = 9900
rsi = 70
latest_closed_candle_open_time = '2023-01-01'

take_profit_price, stop_loss_price = calc_sl_pnl(operation, purchase_price, margin_operation)

if purchased:  # and (operation.startswith('SOBE') or operation.startswith('CAI')):
	perform_sell = False
	if operation == 'SOBE': #self.is_long(operation):
		print('1')
		margin = (actual_price - purchase_price) / purchase_price
		print(f'actual_price: {actual_price} - take_profit_price: {take_profit_price} - stop_loss_price: {stop_loss_price} -  purchase_price: {purchase_price} - margin: {margin} => {actual_price - purchase_price}')
		if ((actual_price >= take_profit_price) or (actual_price <= stop_loss_price)):  # Long ==> Sell - Take Profit / Stop Loss
			print('2')
			perform_sell = True
	elif operation == 'CAI': #self.is_short(operation):
		print('3')
		margin = (purchase_price - actual_price) / purchase_price
		if ((actual_price <= take_profit_price) or (actual_price >= stop_loss_price)):  # Short ==> Sell - Take Profit / Stop Loss
			print('4')
			perform_sell = True

	profit_and_loss = amount_invested * margin
	print(f'Perform Sell: {perform_sell} - Margin: {margin} - PnL: $ {profit_and_loss}')
	if perform_sell:  # Register Sell
		balance += profit_and_loss
		params_operation = utils.get_params_operation(symbol, interval, 'SELL', amount_invested, balance, take_profit_price, stop_loss_price,
																									purchase_price, actual_price, profit_and_loss, rsi, operation)
		#utils.register_operation(params_operation)
		#utils.register_account_balance(balance)
		log_selling(latest_closed_candle_open_time, operation, purchase_price, actual_price, margin, amount_invested, profit_and_loss, balance,
											take_profit_price, stop_loss_price, margin_operation)

In [ ]:
purchased = True
purchase_price = 1000.00
now_price = 900.00
operation = 'CAI_1.5'
purchase_operation = operation
amount_invested = 100.00
stop_loss = 1.5
symbol = 'TESTUSDT'
interval = '1m'
rsi = 70.00

margin = float(operation.split('_')[1])
take_profit = now_price * (1 + margin)
stop_loss = now_price * (1 - (2 * margin))

open_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Sell operation
if purchased:
	diff = 100 * (now_price - purchase_price) / purchase_price
	profit_and_loss = 0.0
	if purchase_operation.startswith('SOBE'):
		profit_and_loss += amount_invested * (diff / 100)
	else:
		profit_and_loss += amount_invested * (-diff / 100)

if (abs(diff) >= stop_loss) and purchased:
	account_balance = utils.get_account_balance()
	balance = account_balance['balance']	
	balance += profit_and_loss
	params_operation = {'operation_date': int(datetime.datetime.now().timestamp() * 1000),
											'symbol': symbol,
											'interval': interval,
											'operation': 'SELL',
											'amount_invested': f'{amount_invested:.2f}',
											'balance': f'{balance:.2f}',
											'take_profit': f'{take_profit:.6f}',
											'stop_loss': f'{stop_loss:.6f}',
											'purchase_price': f'{purchase_price:.6f}',
											'sell_price': f'{now_price:.6f}',
											'PnL': f'{(amount_invested - profit_and_loss):.2f}',
											'rsi': f'{rsi:.2f}',
											'status': operation,
											}

	utils.register_operation(params_operation)
	utils.register_account_balance(balance)

	msg = f'Venda: Symbol: {symbol}_{interval} - open_time: {open_time} - Operação: {purchase_operation} - Valor Comprado: $ {purchase_price:.6f} - \
Valor Venda: $ {now_price:.6f} - Variação: {diff:.2f}% - PnL: $ {(amount_invested-profit_and_loss):.2f} - Balance: $ {balance:.2f}'
	print(msg)
	# Reset variaveis
	purchased = False
	purchase_price = 0.0
	purchase_operation = ''
	amount_invested = 0.0
# End Sell Calcs
print(params_operation)

In [ ]:
import datetime
params_operation = {'operation_date': int(datetime.datetime.now().timestamp() * 1000),
										'symbol': 'TESTUSDT',
										'interval': '1h',
										'operation': 'SELL',
										'amount_invested': 200.00,
										'balance': 9100.11,
										'take_profit': 4342.24,
										'stop_loss': 2323.23,
										'purchase_price': 232323.32,
										'sell_price': 23424.23,
										'PnL': 32424.24,
										'rsi': 43.44,
										'status': 'CAI_2.9',
										}
print(params_operation)
#pd.DataFrame(data=[params_operation], index=[0])
utils.register_operation(params_operation)

In [ ]:
utils.get_account_balance()

In [ ]:
filename = f'{myenv.datadir}/account_balance.dat'
params = {}
params['operation_date'] = int(datetime.datetime.now().timestamp() * 1000)
params['balance'] = 100.99
params['params'] = None
data = pd.DataFrame(data=[params], index=[0])
if os.path.exists(filename):
	base = pd.read_csv(filename, sep=';')
	data = pd.concat([base, data], ignore_index=True)
	data.sort_values('operation_date', inplace=True)
data.to_csv(filename, sep=';', index=False)

In [ ]:
symbol = 'BTCUSDT'
interval = '1h'
amount_invested = 100

params_operation = utils.get_latest_operation(symbol, interval)
if len(params_operation) > 0 and params_operation['operation'] == 'BUY':
	purchased = True
	purchase_price = float(params_operation['purchase_price'])
	take_profit = float(params_operation['take_profit'])
	stop_loss = float(params_operation['stop_loss'])
	purchase_operation = params_operation['status']
	amount_invested = params_operation['amount_invested']
	rsi = 0
	
	open_time = pd.to_datetime(params_operation["operation_date"], unit='ms').strftime('%Y-%m-%d %H:%M:%S')
	msg = f'Compra: Symbol: {symbol} - open_time: {open_time.strftime("%Y-%m-%d %H:%M:%S")} - Operação: {purchase_operation} - Valor Comprado: {purchase_price:.4f} - \
RSI: {rsi:.2f} - PnL: $ {amount_invested:.2f}'
	print(msg)
	print("************ COMPRADO ************")

In [ ]:
res = utils.get_latest_operation('BTCUSDT', '1h')
pd.to_datetime(res['operation_date'], unit='ms').strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
filename = f'{myenv.datadir}/account_balance.dat'
if os.path.exists(filename):
	data = pd.read_csv(filename, sep=';')
	data.sort_values('operation_date', inplace=True)
	print(data.tail(1).to_dict(orient='records')[0])

In [ ]:
t = utils.get_telegram_key()
print(t)
myenv.telegram_key.append(t)

In [ ]:
myenv.telegram_key

In [ ]:
import pandas as pd

# Sample datetime data
date_strings = ["2023-09-24 10:00:00", "2023-09-24 15:30:00"]
dates = pd.to_datetime(date_strings)

# Create a Pandas Series or DataFrame with the datetime index
# Localize it to the Brazil/Brasilia time zone
dates_localized = dates.tz_localize('Brazil/East')

# Print the result
print(dates_localized)

In [ ]:
from binance.client import Client
client = Client()
time = client.get_server_time()
ix = pd.to_datetime(time['serverTime'], unit='ms')
ix

In [ ]:
data = pd.DataFrame(data = client.get_klines(symbol='BTCUSDT', interval='5m', limit=5), columns=myenv.all_klines_cols)
data['open_time'] = pd.to_datetime(data['open_time'], unit='ms')
data['close_time'] = pd.to_datetime(data['open_time'], unit='ms')
data

In [ ]:
data = utils.get_klines(symbol='BTCUSDT', interval='1h', limit=3, max_date=None, columns=myenv.all_klines_cols, parse_data=True)
data

In [ ]:
data.iloc[data.shape[0]-2:data.shape[0]-1]

In [ ]:
data.iloc[0:1]['open_time'].values[0]

In [ ]:
prediction_label = 'CAI_3.99'
print(prediction_label.split('_'))

operation = prediction_label.split('_')[0]
margin = float(prediction_label.split('_')[1])

print(f'operation: {operation} - margin: {margin}')

In [ ]:
date = utils.get_latest_close_time('1h')
date

In [ ]:
actual_value = 100
margin = 1.5

purchase_value = actual_value
take_profit_value = actual_value * (1 + margin/100)
stop_loss_value = actual_value * (1 - margin/100)

print(f'take_profit: {take_profit_value} - stop_loss: {stop_loss_value}')

In [ ]:
#if ix.minute == 0:
ix = ix.replace(hour=ix.hour-1, minute=00, second=00, microsecond=00)
ix

In [ ]:
for i in range(0, 61):
	if  i % 5 == 0:
		print(i)

In [ ]:
ix = ix.replace(hour=14, minute=52, second=32, microsecond=0)
interval = 1

print(ix)
_ix_aux = ix.replace(hour=0, minute=0, second=0, microsecond=0)
day = ix.day-interval
for i in range(day, -1, -1):
	#print(f'i: {i} - min: {min} - rest: {i % interval}')
	if  (i <= day) and(i % interval == 0):
		print(f'hour: {i}')
		_ix_aux = _ix_aux.replace(day=i, hour=0, minute=0, second=0, microsecond=0)
		break
print(f'Last Close Time: {_ix_aux}')


In [ ]:
0 % 30

In [ ]:
data = utils.get_data('BTCUSDT', columns=myenv.all_cols)
data

In [ ]:
data.loc[ix]

In [ ]:
file_list = glob.glob(os.path.join(f'{myenv.datadir}/', 'resultado_simulacao_*.csv'))
top_params = []
columns = None
df_top_params = pd.DataFrame()
for file_path in file_list:
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path, sep=';')
        df_top_params = pd.concat([df_top_params, df.tail(1)], ignore_index=True)


df_top_params.to_csv(f'{myenv.datadir}/top_params.csv', sep=';', index=False)
return df_top_params.to_dict(orient='records')

In [ ]:
import pandas as pd
df = pd.DataFrame({'a': [1,674,3,674], 'b': [3,2,4,1], 'c': [23,35,7,78], 'd': [878,432,66,88]})
df

In [ ]:
df.sort_values(['a', 'b'],  ascending=[False, True])

In [ ]:
df.insert(0, 'dd', [212,4,52,2])
df

In [ ]:
df['tt'] = 'adf,fasd,gasdf,asdf'
df.loc[0, 'tt'] = 'as,f,as,fd,asd,f,asd,f,asd,f,as,df,'
df['c_tt'] = df['tt'].apply(lambda x: len(x.split(',')))
df

In [ ]:
utils.prepare_best_params()

In [ ]:
top_params

In [ ]:
symbol = 'BTCUSDT'
it = '5m'
filename = f'{myenv.datadir}/{symbol}/{symbol}_{it}'
df = pd.read_csv(f'{filename}.csv', sep=';')
# df.info()
df.to_csv(f'{filename}.dat', sep=';', index=False, compression=dict(method='zip'))
print(f'Done! {filename}')

In [ ]:
df_simb = pd.read_csv(f'{myenv.datadir}/symbol_list.csv')
for symbol in df_simb['symbol']:
	for it in ['1h', '5m']:
		filename = f'{myenv.datadir}/{symbol}/{symbol}_{it}'
		df = pd.read_csv(f'{filename}.csv', sep=';')
		# df.info()
		df.to_csv(f'{filename}.dat', sep=';', index=False, compression=dict(method='zip'))
		print(f'Done! {filename}')

In [ ]:
filename = f'{myenv.datadir}/BTCUSDT/BTCUSDT_5m'
df = pd.read_csv(f'{filename}.dat', sep=';', compression=dict(method='zip'))
df.info()
df
#df.to_csv(f'{filename}.csv', sep=';', index=False, compression=dict(method='zip', archive_name=f'{filename}.csv'))

In [ ]:
import datetime
import pandas as pd

now = datetime.datetime.now()
ordinal = int(now.timestamp() * 1000)
print(now)
print(ordinal)
print(pd.to_datetime(ordinal, unit='ms'))

In [ ]:
numeric_features = 'open,close,volume'
date_features = ['open_time']

kline_features = date_features + numeric_features.split(',')

print(kline_features)

In [ ]:
import numpy as np
from datetime import datetime
import pandas as pd
import pytz

date_np = np.datetime64('2023-10-04T20:00:00.000000000')
print(f'{date_np} - {type(date_np)} - {datetime.fromtimestamp(int(date_np.astype(np.int64))/1000000000)} - {type(datetime.fromtimestamp(int(int(date_np)/1000000000)))}')

In [ ]:
from binance.client import Client
from binance.streams import BinanceSocketManager

client = Client()

bm = BinanceSocketManager(client)
def process_m_message(msg):
	print("stream: {} data: {}".format(msg['stream'], msg['data']))

conn_key = bm.start_multiplex_socket(['sxpusdt@aggTrade', 'sxpusdt@ticker','sxpusdt@markPrice'], process_m_message)
bm.start()

In [ ]:
from binance.client import Client

client = Client()

In [ ]:
import pandas as pd

pd.to_datetime(1502942400000, unit='ms')

In [ ]:
client.get_historical_klines(symbol='BTCUSDT', interval=client.KLINE_INTERVAL_1MINUTE, start_str='2010-01-01', end_str='2017-12-31', limit=1000)

In [ ]:
from itertools import combinations
text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
for size in range(1, len(numeric_features) + 1): 
	comb = map(list, combinations(numeric_features, size))	
	for c in comb:
		res = ''
		for j in c:
			res += f'{j},'
		combination_numeric_features.append(res[0:len(res) - 1])

print(f'Combination Numeric Features: \n{len(combination_numeric_features)} ')

In [ ]:
import pandas as pd
symbol = 'BTCUSDT'
interval = '1h'
estimator = 'knn'
fix_imbalance_method = 'instancehardnessthreshold'
features = 'open_time,status,volume,taker_buy_base_asset_volume,rsi'.split(',')
print(str(features))

simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'
df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')
#df_resultado_simulacao.info()
print(df_resultado_simulacao.shape)

chave = (df_resultado_simulacao['symbol'] == symbol) & \
    (df_resultado_simulacao['interval'] == interval) & \
    (df_resultado_simulacao['estimator'] == estimator) & \
    (df_resultado_simulacao['fix_imbalance_method'] == fix_imbalance_method) & \
		(df_resultado_simulacao['features'] == str(features))
print(chave.sum())

In [ ]:
import pandas as pd

data1 = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
data2 = pd.DataFrame({'c': [55, 66], 'd': [77, 88]})

data3 = pd.concat([data1, data2], axis=1)

data3

In [ ]:
data1['a'] * data2['d']

In [1]:
import sys
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
import src.myenv as myenv
import src.send_message as sm
import os
import glob
import pandas as pd

In [2]:
def set_status_PL(row, stop_loss, max_regression_profit_and_loss, prefix_col_diff, strategy='SOBE_CAI'):
  for s in range(1, max_regression_profit_and_loss + 1):
    if (strategy == 'SOBE') or (strategy == 'SOBE_CAI'):
      if row[f'{prefix_col_diff}{s}'] >= stop_loss:
        return f'SOBE_{stop_loss}'
    if (strategy == 'CAI') or (strategy == 'SOBE_CAI'):
      if row[f'{prefix_col_diff}{s}'] <= -stop_loss:
        return f'CAI_{stop_loss}'
  return 'ESTAVEL'

In [10]:
def reg_th(_data, label, diff_percent, max_regression_profit_and_loss, diff_col, strategy):
	if strategy == 'SOBE_CAI':
		_data[label + '_sobe'] = _data.apply(set_status_PL, axis=1, args=[diff_percent, max_regression_profit_and_loss, diff_col, 'SOBE'])
		_data[label + '_cai'] = _data.apply(set_status_PL, axis=1, args=[diff_percent, max_regression_profit_and_loss, diff_col, 'CAI'])
		_data[label + '_sobe'] = pd.Categorical(_data[label + '_sobe'])
		_data[label + '_cai'] = pd.Categorical(_data[label + '_cai'])
		return _data
	else:
		_data[label] = _data.apply(set_status_PL, axis=1, args=[diff_percent, max_regression_profit_and_loss, diff_col, 'SOBE_CAI'])
		_data[label] = pd.Categorical(_data[label])
		return _data

In [11]:
def regression_PnL(data: pd.DataFrame, label: str, diff_percent: float, max_regression_profit_and_loss=6, drop_na=True, drop_calc_cols=True, strategy=None):
  col = 'c_'
  diff_col = 'd_'
  cols = []
  diff_cols = []
  _data = pd.DataFrame()
  for i in range(1, max_regression_profit_and_loss + 1):
    _data[col + str(i)] = data['close'].shift(-i)
    _data[diff_col + str(i)] = 100 * ((_data[col + str(i)] - data['close']) / data['close'])
    cols.append(col + str(i))
    diff_cols.append(diff_col + str(i))

  _data = reg_th(_data, label, diff_percent, max_regression_profit_and_loss, diff_col, strategy)

  if drop_calc_cols:
    _data.drop(columns=cols + diff_cols, inplace=True)

  data = pd.concat([data, _data], axis=1)
  if drop_na:
    data.dropna(inplace=True)

  return data

In [12]:
import src.utils as utils

data = utils.get_data(symbol='BTCUSDT', interval='1h', tail=-1, parse_dates=True, columns=myenv.all_cols)
data.info()
data = regression_PnL(data, 'close', 1.5, 6, drop_na=True, drop_calc_cols=True, strategy='SOBE_CAI')
data.info()
data

<built-in method __dir__ of sqlite3.Connection object at 0x7f546555ab40>
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53959 entries, 2017-08-17 04:00:00 to 2023-10-18 17:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     53959 non-null  datetime64[ns]
 1   open                          53959 non-null  float32       
 2   high                          53959 non-null  float32       
 3   low                           53959 non-null  float32       
 4   close                         53959 non-null  float32       
 5   volume                        53959 non-null  float32       
 6   close_time                    53959 non-null  int64         
 7   quote_asset_volume            53959 non-null  float32       
 8   number_of_trades              53959 non-null  int16         
 9   taker_buy_base_asset_volume   53959 non-null  float3

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,close_sobe,close_cai
ix_open_time,,,,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,4261.479980,4313.620117,4261.319824,4308.830078,47.181007,1502945999999,2.023661e+05,171,35.160503,1.509525e+05,BTCUSDT,SOBE_1.5,ESTAVEL
2017-08-17 05:00:00,2017-08-17 05:00:00,4308.830078,4328.689941,4291.370117,4315.319824,23.234917,1502949599999,1.003048e+05,102,21.448071,9.260828e+04,BTCUSDT,SOBE_1.5,ESTAVEL
2017-08-17 06:00:00,2017-08-17 06:00:00,4330.290039,4345.450195,4309.370117,4324.350098,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04,BTCUSDT,SOBE_1.5,ESTAVEL
2017-08-17 07:00:00,2017-08-17 07:00:00,4316.620117,4349.990234,4287.410156,4349.990234,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04,BTCUSDT,SOBE_1.5,ESTAVEL
2017-08-17 08:00:00,2017-08-17 08:00:00,4333.319824,4377.850098,4333.319824,4360.689941,0.972807,1502960399999,4.239503e+03,28,0.814655,3.552747e+03,BTCUSDT,SOBE_1.5,ESTAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-18 13:00:00,2023-10-18 13:00:00,28330.009766,28384.650391,28261.009766,28325.630859,1459.262573,1697637599999,4.134159e+07,-3211,669.374268,1.896345e+07,BTCUSDT,ESTAVEL,ESTAVEL
2023-10-18 14:00:00,2023-10-18 14:00:00,28325.619141,28362.699219,28142.869141,28190.490234,2948.973145,1697641199999,8.332144e+07,32756,1192.909546,3.369884e+07,BTCUSDT,ESTAVEL,ESTAVEL
2023-10-18 15:00:00,2023-10-18 15:00:00,28190.490234,28285.609375,28157.830078,28247.449219,1196.635376,1697644799999,3.378677e+07,-11970,656.642517,1.854051e+07,BTCUSDT,ESTAVEL,ESTAVEL
